# Scraping posts from r/conspiracy

Extraction of all the posts resent in r/conspiracy, with title, content and url.
First of all we need to import praw and access reddit:

In [10]:
import praw
reddit = praw.Reddit(client_id = "", client_secret= "", user_agent= "")

We can now access the subreddit, extract all the posts in it adn check which other subreddit reposted that posts and how much:

In [12]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

#put the subreddit already passed here
id_to_avoid = ["conspiracy"]

for i in reddit.subreddit("conspiracy").top(limit=500):
    post_list.append((i.title, i.score, "https://www.reddit.com"+i.permalink))


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]):
        if repost.subreddit_id not in id_to_avoid:
            subreddit_url = str(repost.subreddit)
            subreddit_url = "https://www.reddit.com/r/" + subreddit_url
            if subreddit_url in conspiracy_dict.keys():
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1][0] +=1
            else:
                conspiracy_dict[subreddit_url]=[[], []]
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1].append(1)


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_top.csv',index=False)




In [17]:
print(conspiracy_dict)


{'https://www.reddit.com/r/TopMindsOfReddit': [['https://www.reddit.com/r/TopMindsOfReddit/comments/ctpf6s/rconspiracy_china_is_censoring_reddit_they_remove/', 'https://www.reddit.com/r/TopMindsOfReddit/comments/coi2d7/you_know_im_gonna_give_top_minds_this_one/', 'https://www.reddit.com/r/TopMindsOfReddit/comments/coit6q/top_mind_has_some_top_advice_for_the_children_and/', 'https://www.reddit.com/r/TopMindsOfReddit/comments/bb2ywp/top_conspiratards_claim_reddit_is_censoring_again/', 'https://www.reddit.com/r/TopMindsOfReddit/comments/hpr4dq/top_nazis_of_rconspiracy_discuss_jews_they_dont/', 'https://www.reddit.com/r/TopMindsOfReddit/comments/98dft6/top_minds_have_been_confronted_with_evidence_of/', 'https://www.reddit.com/r/TopMindsOfReddit/comments/itaied/daddy_gave_us_hemp_and_1200_beta_bucks_why_does/', 'https://www.reddit.com/r/TopMindsOfReddit/comments/dfknjn/weve_gone_off_the_deep_end_of_the_rainbow_ive/', 'https://www.reddit.com/r/TopMindsOfReddit/comments/df2pyh/brave_rconspira